# Mortgage Calculator

Graphical interface using ipywidgets to calculate the evolution of principal and interest by varying interest rate, lump sum payment, compounding frequency, payment, and payment frequency. 

Useful for **variable rate** mortgages to determine what payment is needed to pay off principal over the amortization period provided that rates remain at their current level. If this is the case, use the 'term_start' variable to control when you actually started the loan so that the term and amortization period is correct.

Note: For full freedom in adjusting payment frequency, need to use daily (D) compounding frequency. Default is month-start (MS) payments with MS interest compounding. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

import ipywidgets as widgets
from ipywidgets import FloatSlider, IntSlider

from IPython.display import display

%matplotlib inline

In [2]:
widgets.__version__

'7.6.5'

In [3]:
def monthly_to_daily(r):
    return np.power(1 + r, 1/(365/12)) - 1

In [4]:
def setup_data(principal=100000, start=None, end=None, freq='d', payment=2000, payment_freq='MS', r=0.):
    rng = pd.date_range(start, end, freq=freq)   
    data = pd.DataFrame(np.nan, index=rng, columns=['principal', 'interest', 'payment', 'isum', 'ipayment', 'ppayment'])  
    pymnt = data['payment'].asfreq(payment_freq)
    pymnt[:] = payment
    
    data.loc[pymnt.index, 'payment'] = payment
    data['payment'] = data['payment'].fillna(0)
    
    data.iloc[0] = [principal, 0, 0, 0, np.nan, np.nan]

    return data

In [5]:
def iterate_principal(data, r=0., freq='MS'):
    period_isum = 0
    for i in range(1, len(data.index)):
        dt = data.index[i]
        x0 = data.iloc[i-1]
        x1 = data.iloc[i]
        
        p0 = x0['principal']
        pay = x1['payment']
        interest = p0 * r
        
        data.loc[dt, 'principal'] = p0 + interest - pay
        data.loc[dt, 'interest'] = interest
        data.loc[dt, 'isum'] = x0['isum'] + interest
        
        period_isum += interest
        
        if pay > 0:
            data.loc[dt, 'ipayment'] = period_isum
            data.loc[dt, 'ppayment'] = pay - period_isum
            period_isum = 0

    return data

In [6]:
import matplotlib.ticker as ticker

def dollarFormatter(scale=1000.):
    @ticker.FuncFormatter
    def wrap(x, pos):
        x = x / scale

        suffix = ''
        if scale == 1000:
            suffix = 'K'

        return '${x:.0f}{suffix}'.format(x=x, suffix=suffix)
    return wrap

In [7]:
@widgets.interact(
    payment=FloatSlider(min=0, max=5e3, step=1e-1, value=2000, continuous_update=False),
    principal=FloatSlider(min=100000, max=600000, step=1, value=500000, continuous_update=False),
    annual_rate=FloatSlider(min=0, max=10, step=1e-1, value=1.5, continuous_update=False),
    lump_sum=FloatSlider(min=0, max=100000, step=1, value=0, continuous_update=False),
    term=IntSlider(min=1, max=10, value=5, continuous_update=False),
    amort=IntSlider(min=1, max=40, value=25, continuous_update=False),
)
def run(lump_sum=0, principal=500000, annual_rate=1.5, start=date.today().strftime('%Y-%m-%d'), term=5, 
        amort=25, payment=2000, payment_freq='MS', term_start='2021-08-01', compound_freq='MS'):
    # Output result to global var
    global data
    
    # Rate usually quoted as the annualized monthly rate,
    # e.g. i.e. monthly rate multiplied by 12
    rate = annual_rate / 12 / 100
    
    if compound_freq not in ['MS', 'D']:
        raise Exception('Only MS (month-start) D (daily) frequency supported.')
    
    if compound_freq == 'D':
        rate = monthly_to_daily(rate)
    
    p = principal - lump_sum
    
    start = pd.to_datetime(start)
    term_start = pd.to_datetime(term_start)
    
    term_end = term_start + pd.DateOffset(years=term)
    amort_end = term_start + pd.DateOffset(years=amort)
    
    # Setup
    data = setup_data(principal=p, start=start, end=amort_end, 
                      freq=compound_freq.upper(), payment=payment, payment_freq=payment_freq)
    
    # Iterate
    data = iterate_principal(data, r=rate, freq=payment_freq)
    
    # Plots
    fig, axes = plt.subplots(2, 2, figsize=(8,6))
    ax = axes.flatten()
    fig.subplots_adjust(wspace=0.3)

    # Plot principal over time
    data['principal'].plot(ax=ax[0], label='Principal')
    ax[0].axvline(term_end, ls='--', label='Term End', lw=0.5)
    ax[0].axhline(principal, color='r', ls='--', lw=0.5)
    ax[0].axhline(0, color='g', ls='--', lw=0.5)
    ax[0].yaxis.set_major_formatter(dollarFormatter(scale=1000))
    ax[0].legend()
    
    # Plot total interest over time
    data['isum'].plot(ax=ax[1], label='Total Interest')
    ax[1].axvline(term_end, ls='--', label='Term End', lw=0.5)
    ax[1].axhline(principal, color='r', ls='--', lw=0.5)
    ax[1].axhline(0, color='g', ls='--', lw=0.5)
    ax[1].yaxis.set_major_formatter(dollarFormatter(scale=1000))
    ax[1].legend()
    
    # Plot principal paid per period
    data['ppayment'].dropna().plot(ax=ax[2], label='Principal Payment')
    ax[2].axvline(term_end, ls='--', label='Term End', lw=0.5)
    ax[2].axhline(payment, color='g', ls='--', lw=0.5)
    ax[2].axhline(0, color='r', ls='--', lw=0.5)
    ax[2].yaxis.set_major_formatter(dollarFormatter(scale=1))
    ax[2].legend()
    
    data['ipayment'].dropna().plot(ax=ax[3], label='Interest Payment')
    ax[3].axhline(payment, ls='--', color='r', lw=0.5)
    ax[3].axvline(term_end, ls='--', label='Term End', lw=0.5)
    ax[3].axhline(0, color='g', ls='--', lw=0.5)
    ax[3].yaxis.set_major_formatter(dollarFormatter(scale=1))
    ax[3].legend()
    
    plt.show()
    
    display(data.head())
    display(data.tail())

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='lump_sum', max=100000.0, st…